In [ ]:
# top-level requirements: pytorch-forecasting==0.10.3, citylearn==1.7, ipykernel, jupyter, tensorboard, plotly
# install numba for improved speed
# to use cuda support, run: pip3 install torch==1.13 --index-url https://download.pytorch.org/whl/cu117 - see https://pytorch.org/get-started/locally/

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from models.TFT.TFT_predictor import TFT_Predictor

In [ ]:
train_path = os.path.join('data','example','train')
val_path = os.path.join('data','example','validate')
with open(os.path.join(train_path,'metadata_ext.json')) as json_file:
    UCam_ids = json.load(json_file)["UCam_building_ids"]

---

In [ ]:
TFT_pred = TFT_Predictor(model_group_name='test',load=False)

---

Test model creation and training

In [ ]:
model_type = 'load'
model_name = 'load_test'
i = 0
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)

In [ ]:
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

---

Try loading the model group

In [ ]:
TFT_pred = TFT_Predictor(model_group_name='test',model_names=UCam_ids)

In [ ]:
print(TFT_pred.model_names)

---

In [ ]:
# create and train load models
model_type = 'load'
for i,id in enumerate(UCam_ids):
    model_name = f'{model_type}_{id}'
    train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
    tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
    TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train solar models
model_type = 'solar'
for i,id in enumerate(UCam_ids):
    model_name = f'{model_type}_{id}'
    train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
    tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
    TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train pricing model
model_type = 'pricing'
model_name = 'pricing'
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train carbon model
model_type = 'carbon'
model_name = 'carbon'
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

---

Test prediction inference

In [ ]:
from citylearn.citylearn import CityLearnEnv
env = CityLearnEnv(os.path.join(val_path,'schema.json'))

In [ ]:
env.reset()
for t in range(1000): env.step(np.zeros((len(env.buildings),1)))

In [ ]:
# construct base df with time & past weather info
months = env.buildings[0].energy_simulation.month[env.time_step-TFT_pred.L+1:env.time_step+TFT_pred.T+1]
hours = env.buildings[0].energy_simulation.hour[env.time_step-TFT_pred.L+1:env.time_step+TFT_pred.T+1]
day_types = env.buildings[0].energy_simulation.day_type[env.time_step-TFT_pred.L+1:env.time_step+TFT_pred.T+1]
day_save_statuses = env.buildings[0].energy_simulation.daylight_savings_status[env.time_step-TFT_pred.L+1:env.time_step+TFT_pred.T+1]
past_temps = env.buildings[0].weather.outdoor_dry_bulb_temperature[env.time_step-TFT_pred.L+1:env.time_step+1]
past_dif_irads = env.buildings[0].weather.diffuse_solar_irradiance[env.time_step-TFT_pred.L+1:env.time_step+1]
past_dir_irads = env.buildings[0].weather.direct_solar_irradiance[env.time_step-TFT_pred.L+1:env.time_step+1]

base_df = pd.DataFrame({
    'Month': months,
    'Hour': hours,
    'Day Type': day_types,
    'Daylight Savings Status': day_save_statuses,
    TFT_pred.temp_col_name: np.append(past_temps,np.zeros(TFT_pred.T)),
    TFT_pred.dif_irad_col_name: np.append(past_dif_irads,np.zeros(TFT_pred.T)),
    TFT_pred.dir_irad_col_name: np.append(past_dir_irads,np.zeros(TFT_pred.T))
})
base_df = TFT_pred.reformat_df(base_df,'pred',TFT_pred.time_varying_known_categoricals)

In [ ]:
# perform load forecasting
predicted_loads = []
for j,model in enumerate(TFT_pred.models['load'].values()):

    # construct data df 
    data_df = base_df.copy()
    data_df[TFT_pred.load_col_name] = np.append(env.buildings[j].energy_simulation.non_shiftable_load[env.time_step-TFT_pred.L+1:env.time_step+1],np.zeros(TFT_pred.T))

    # perform prediction
    load_prediction = np.array(model.predict(data_df, mode='prediction')).reshape(TFT_pred.T)[:24]
    # TODO: can I simply take in a dataframe for prediction or do I need to convert it to a dataloader or TimeSeriesDataSet?

    # save prediction in structure
    predicted_loads.append(load_prediction)

In [ ]:
print(np.array(predicted_loads).shape,np.array(predicted_loads))

In [ ]:
import warnings

In [ ]:
TFT_pred.initialise_forecasting(tau=48)
obs = env.reset()

with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore',module=r'pytorch_forecasting')
    for t in range(76):
        forecasts = TFT_pred.compute_forecast(obs, env)
        if forecasts is not None: print(forecasts)
        obs, _, done, _ = env.step(np.zeros((len(env.buildings),1)))

In [ ]:
print(TFT_pred.model_names)